In [1]:
import os
from data.source.pg_experiment import get_pg_experiment_dataframe
import polars as pl
from models.SimpleCNN import PronunciationDataset, collate_fn, SimpleCNN, train, evaluate
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
from path import RESULT_DIRECTORY
import wandb, wandb_config

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\fisch\_netrc
wandb: Currently logged in as: fischbach-kamil (fischbach-kamil-pg) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [2]:
df_pron, df_tone = get_pg_experiment_dataframe(".ogg")


df_stageI_polish = df_pron.filter((pl.col("stage") == 1) & (pl.col("mother") == "polish"))

df_other_lang = df_pron.filter(
    (pl.col("stage") == 1) & ~(pl.col("mother") == "polish")
)

df_stageI_polish


Missing files in mother language Polish: 0
Missing files in mother language other than Polish: 2


shape: (2, 7)
┌────────────┬───────┬─────────┬───────────────────┬───────────┬────────┬─────────┐
│ id_student ┆ value ┆ word_id ┆ rec_path          ┆ univ      ┆ gender ┆ mother  │
│ ---        ┆ ---   ┆ ---     ┆ ---               ┆ ---       ┆ ---    ┆ ---     │
│ i64        ┆ i64   ┆ str     ┆ str               ┆ str       ┆ str    ┆ str     │
╞════════════╪═══════╪═════════╪═══════════════════╪═══════════╪════════╪═════════╡
│ 757        ┆ 1     ┆ a0      ┆ stageI/757/a0.wav ┆ CLES_UMK2 ┆ f      ┆ chinese │
│ 757        ┆ 1     ┆ a1      ┆ stageI/757/a1.wav ┆ CLES_UMK2 ┆ f      ┆ chinese │
└────────────┴───────┴─────────┴───────────────────┴───────────┴────────┴─────────┘


In [5]:
import polars as pl
import numpy as np
from typing import Tuple

def stratified_split(df: pl.DataFrame, label_col: str, train_frac=0.8, val_frac=0.1, seed=42) -> Tuple[pl.DataFrame, pl.DataFrame, pl.DataFrame]:
    classes = df.select(label_col).unique().to_series()
    train_rows, val_rows, test_rows = [], [], []

    rng = np.random.RandomState(seed)

    for cls in classes:
        class_df = df.filter(pl.col(label_col) == cls)
        n = class_df.height
        indices = rng.permutation(n)

        train_end = int(train_frac * n)
        val_end = int((train_frac + val_frac) * n)

        train_rows.append(class_df[indices[:train_end]])
        val_rows.append(class_df[indices[train_end:val_end]])
        test_rows.append(class_df[indices[val_end:]])

    train_df = pl.concat(train_rows)
    val_df = pl.concat(val_rows)
    test_df = pl.concat(test_rows)

    return train_df, val_df, test_df


In [6]:

# dataset = PronunciationDataset(df_stageI_polish)
dataset_other = PronunciationDataset(df_other_lang)

In [11]:
from dataset import Cast, TorchDataset
from functools import partial

from transformation import TorchVadLogMelSpec, TorchVadMFCC
from develop import reload_function

reload_function(TorchVadMFCC)

train_pl, val_pl, test_pl = stratified_split(df_stageI_polish, label_col="value", train_frac=0.6, val_frac=0.2)

to_dataset = lambda dataframe: TorchDataset(
    Cast(dataframe.get_column("rec_path"), TorchVadMFCC(n_mfcc=20)), 
    Cast(dataframe.get_column("value"), lambda x: torch.tensor(x)),
)
dataset_train = to_dataset(train_pl)
dataset_val = to_dataset(val_pl)
dataset_test = to_dataset(test_pl)

In [ ]:
from pytorch_dataloader import MemoryLoadedDataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_loader = DataLoader(dataset_train, batch_size=16, shuffle=True, collate_fn=collate_fn, num_workers=4)
val_loader = DataLoader(dataset_val, batch_size=16, shuffle=False, collate_fn=collate_fn, num_workers=4)
test_loader = DataLoader(dataset_test, batch_size=16, shuffle=False, collate_fn=collate_fn, num_workers=4)

train_loader = MemoryLoadedDataLoader(train_loader, device=device)
print("Loaded train loader into memory")
val_loader = MemoryLoadedDataLoader(val_loader, device=device)
print("Loaded validation loader into memory")

Loaded train loader into memory
Loaded validation loader into memory


In [11]:


lr=1e-3
pth = "SimpleCNN.pth"

# Start a new wandb run to track this script.
run = wandb.init(
    # name of the run
    name="SimpleCNN-run-2",
    config={
        "Name": SimpleCNN,
        "learning_rate": lr ,
        "optimizer": "Adam",
        "criterion": "BCELoss",
        "architecture": "SimpleCNN",
        "dataset": "Stage-I-only-polish",
        "train_val_test(%)": "60-20-20",
        "epochs": 5,
    },
)

model = SimpleCNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.BCELoss()

epochs = 5
for epoch in range(epochs):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = evaluate(model, val_loader, criterion, device)
    # Logging the metadata for each epoch so that the charts can be generated on the dashboard
    run.log({"train_acc": train_acc, "train_loss": train_loss, "val_acc": val_acc, "val_loss": val_loss, })
    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

run.log({"model_eval": model.eval()})
# Saving the model to pth and adding it to the artifacts of the run, there is 5GB of memory on wandb so we should be fine.
torch.save(model.state_dict(), os.path.join(RESULT_DIRECTORY, pth))
artifact = wandb.Artifact("simplecnn-model", type="model")
artifact.add_file(os.path.join(RESULT_DIRECTORY, pth))
run.log_artifact(artifact)

# Finish the run so it gets send to the remote. You can discover the run right after that on the dashboard.
run.finish()


Epoch 1, Train Loss: 0.6825, Train Acc: 0.5810, Val Loss: 0.6681, Val Acc: 0.5908
Epoch 2, Train Loss: 0.6633, Train Acc: 0.5978, Val Loss: 0.6837, Val Acc: 0.5794
Epoch 3, Train Loss: 0.6473, Train Acc: 0.6217, Val Loss: 0.6575, Val Acc: 0.6195
Epoch 4, Train Loss: 0.6359, Train Acc: 0.6263, Val Loss: 0.6491, Val Acc: 0.6265
Epoch 5, Train Loss: 0.6264, Train Acc: 0.6455, Val Loss: 0.6469, Val Acc: 0.6274


train_acc,▁▃▅▆█
train_loss,█▆▄▂▁
val_acc,▃▁▇██
val_loss,▅█▃▁▁
model_eval,SimpleCNN( (conv1)...
train_acc,0.64554
train_loss,0.62637
val_acc,0.6274
val_loss,0.64694


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_saved = SimpleCNN()
model_saved.load_state_dict(torch.load(os.path.join(RESULT_DIRECTORY, "initial_model_t&val_080101split.pth")))
model_saved.to(device)
model_saved.eval()

SimpleCNN(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=16384, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=1, bias=True)
)